In [10]:
import pandas as pd
import json
import os
import glob
from pprint import pprint

In [11]:
# read the data from the json files
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
target_dir = os.path.join(parent_dir,'stock_scrape_load','data')
file_path = f"{target_dir}/*.json"

# declare empty list to store data
data = []

# Loop through all json files and update data list
for file in glob.glob(file_path):
    with open(file, 'r') as json_file:
        json_data = json.load(json_file)
        data.append(json_data)

# Create Dataframe
prices_df = pd.concat([pd.DataFrame(d) for d in data], ignore_index = True)


In [4]:
# View that sweet sweet compliled dataframe!
prices_df = prices_df.sort_values(['ticker','date'])
prices_df.head()


,ticker,date,sector,52WeekChange,ask,bid,52WeekHigh,52WeekLow,dailyOpen,previousClose,dailyVolume,quickRatio,10DayAverageVolume,marketCap,dayHigh,dayLow
100,AAPL,2023-07-11,Technology,29.273487,188.24,188.12,194.48,124.17,189.16,188.61,46509718.0,0.764,51012280.0,2958253883392.0,NaN,NaN
200,AAPL,2023-07-12,Technology,29.273487,0.0,0.0,194.48,124.17,189.68,188.08,26845497.0,0.764,51012280.0,2970051936256.0,NaN,NaN
0,AAPL,2023-07-13,Technology,27.81707,190.33,190.32,194.48,124.17,190.5,189.77,38321242.0,0.764,52013490.0,2996946599936.0,NaN,NaN
300,AAPL,2023-07-13,None,None,None,None,None,None,None,None,None,None,None,None,None,None
400,AAPL,2023-07-14,Technology,26.882862,190.01,190.0,194.48,124.17,190.23,190.54,21271537.0,0.764,51027070.0,2984992833536.0,191.175,189.725


In [8]:
# Pick out the columns I want included in final json
ticker = pd.DataFrame(prices_df[['ticker','date','ask','bid','dailyOpen','previousClose','dailyVolume']])

print(f"There are {len(prices_df.value_counts(['date']))} days of data represented")
ticker.head()


There are 4 days of data represented


,ticker,date,ask,bid,dailyOpen,previousClose,dailyVolume
100,AAPL,2023-07-11,188.24,188.12,189.16,188.61,46509718.0
200,AAPL,2023-07-12,0.0,0.0,189.68,188.08,26845497.0
0,AAPL,2023-07-13,190.33,190.32,190.5,189.77,38321242.0
300,AAPL,2023-07-13,None,None,None,None,None
400,AAPL,2023-07-14,190.01,190.0,190.23,190.54,21271537.0


In [7]:
## Old code-- delete later

# prices_df.to_json(
#     path_or_buf = 'all_prices_xdf.json',
#     orient = 'records',
#     date_format = 'iso')

# Old code - second attempt - delete later

In [9]:
historical_json = []
grouped_tickers = prices_df.groupby(['ticker'])
grouped_dates = prices_df.groupby(['date'])
data = {
    "tickers": list(grouped_tickers.groups.keys()),
    "dates": list(grouped_dates.groups.keys()),
    "history": []
}

for ticker, group in grouped_tickers:
    ticker_data = {
        "ticker": ticker,
        "stats": []
    }
    
    for date, day_group in group.groupby('date'):
        date_data ={
            "date": date,
            "market" : []
        }
    
        for index, row in day_group.iterrows():
            entry_data = {
                # "symbol": row['ticker'],
                # "date": row['date'],
                "dailyOpen": row['dailyOpen'],
                "ask": row['ask'],
                "bid": row['bid'],
                "previousClose": row['previousClose'],
                "dailyVolume": row['dailyVolume']
            }
            date_data["market"].append(entry_data)
        ticker_data["stats"].append(date_data)
    data["history"].append(ticker_data)
    #     ticker_data["day"].append(day_data)
    # data["days"].append(ticker_data)

json_data = json.dumps(data, indent=4)
with open("historical.json", "w") as json_file:
    json_file.write(json_data)

In [9]:
# Old code - delete later

#  THe follow is an attempt to reorganize the datafram into a structured json.  Then I realized it did not really matter
# json_data = {}

# for index, row in prices_df.iterrows():
#     date = row['date']
#     open = row['dailyOpen'], 
#     close = row['previousClose'], 
#     volume = row['dailyVolume'],
#     ticker = row['ticker'], 
#     cap = row['marketCap'], 
#     quick = row['quickRatio'],
#     sector = row['sector'],
#     low = row['52WeekLow'], 
#     high = row['52WeekHigh'], 
#     change = row['52WeekChange'],
#     ave_vol = row['10DayAverageVolume']

#     if date not in json_data:
#         json_data[date] = {'price':[],'info':[],'stats':[]}

#     json_data[date]['price'].append({'ticker':ticker,
#                                      'dailyOpen':open,
#                                      'previousClose':close,
#                                      'dailyVolume':volume
#                                                       })
#     json_data[date]['info'].append({'sector':sector,
#                                     'marketCap':cap,
#                                     })
#     json_data[date]['stats'].append({'52WeekLow':low,
#                                      '52WeekHigh':high, 
#                                      '52WeekChange': change,
#                                      '10DayAverageVolume':ave_vol
#                                      })
# # price_json = json.dumps(json_data)

# # pprint(price_json)

# prices = pd.DataFrame(json_data)
# prices.to_json('all_prices.json')


# Old code --  delet later

# Same commetn as previous cell

# output_path = f"{target_dir}/all_prices.json"

# with open('all_prices.json','w') as file:
#     file.write(price_json)